In [1]:
from pyomo.environ import *
model = ConcreteModel()

In [2]:
grapes = ['gSB11','gSL10','gSL11','gSB10']
wines = ['wSB11','wSL10','wSL11','wCab','wMer']

In [3]:
model.x = Var(grapes,wines,domain=NonNegativeReals)

In [5]:
gYear = {}
gYear['gSB11'] = '2011'
gYear['gSL10'] = '2010'
gYear['gSL11'] = '2011'
gYear['gSB10'] = '2010'



3 Set Declarations
    x_index : Dim=0, Dimen=2, Size=20, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['gSB10', 'gSB11', 'gSL10', 'gSL11']
    x_index_1 : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=None
        ['wCab', 'wMer', 'wSB11', 'wSL10', 'wSL11']

1 Var Declarations
    x : Size=20, Index=x_index
        Key                : Lower : Value : Upper : Fixed : Stale : Domain
         ('gSB10', 'wCab') :     0 :  None :  None : False :  True : NonNegativeReals
         ('gSB10', 'wMer') :     0 :  None :  None : False :  True : NonNegativeReals
        ('gSB10', 'wSB11') :     0 :  None :  None : False :  True : NonNegativeReals
        ('gSB10', 'wSL10') :     0 :  None :  None : False :  True : NonNegativeReals
        ('gSB10', 'wSL11') :     0 :  None :  None : False :  True : NonNegativeReals
         ('gSB11', 'wCab') :     0 :  None :  None : False :  True : Non